In [1]:
### Router Engine

In [2]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
###Load data

In [5]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [7]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [8]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [9]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [10]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [11]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

In [12]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [13]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document introduces MetaGPT, a meta-programming framework that enhances multi-agent collaboration based on Large Language Models (LLMs) by incorporating Standardized Operating Procedures (SOPs) inspired by human workflows. MetaGPT aims to streamline workflows by assigning specific roles to agents, breaking down complex tasks into subtasks, and ensuring structured communication among agents. It emphasizes the importance of role specialization, workflow efficiency, and executable feedback mechanisms to improve code generation quality. The framework achieves state-of-the-art performance in evaluations and offers a promising approach for developing LLM-based multi-agent systems with enhanced robustness and efficiency. Additionally, the document discusses the development process using MetaGPT for software development, outlining the roles of different agents in the software development lifecycle and the perf

In [14]:
print(len(response.source_nodes))

34


In [15]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is more relevant as it focuses on retrieving specific context from the MetaGPT paper, which may provide insights on how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool where they can publish structured messages. This shared message pool allows all agents to exchange messages directly, enabling them to both publish their own messages and access messages from other entities transparently. Agents can retrieve required information directly from this shared pool, eliminating the need to inquire about other agents and wait for their responses, thus enhancing communication efficiency.


In [16]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [17]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: The ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results demonstrate the effectiveness of MetaGPT in addressing challenges related to information overload and reducing hallucinations in software generation tasks. By utilizing a global message pool and a subscription mechanism, MetaGPT successfully streamlines communication and filters out irrelevant contexts, enhancing the relevance and utility of information. This design approach helps in improving the performance of the system by focusing on granular tasks and maintaining information validity in lengthy contexts.
